In [26]:
import math
import numpy as np

In [339]:
class GraphNet:
    # Make an empty graph on n nodes with linear activation
    def __init__(self, n):
        self.size = n
        self.adj = np.zeros((n, n))
        self.weights = np.zeros((n, n))
        self.act = ['linear' for i in range(self.size)]
        self.input_nodes = []
        self.output_nodes = []
        self.tempstates = [[0 for i in range(self.size)]]
        self.states = [[0 for i in range(self.size)]]
        self.mode = 'overwrite'
        self.time = 0
    
    # Sets the mode for the step method
    # Options: overwrite, add
    def mode(self, m):
        self.mode = m
    
    # Connect an edge from node i to node j
    def connect(self, i, j, w=1):
        self.adj[j,i] = w
    
    # Change activation on node i
    # Options: linear, relu, sigmoid/logistic, tanh
    def activation(self, i, f):
        self.act[i] = f
    
    # Get the list of edge weights that are inputs to node i
    def inputs(self, i):
        ins = []
        for k in range(self.size):
            if self.adj[i,k] != 0:
                ins.append(k)
        return ins
        #return np.ndarray.tolist(np.ndarray.astype(self.adj[i,:], int))
    
    # Get the list of edge weights that are outputs from node i
    def outputs(self, i):
        outs = []
        for k in range(self.size):
            if self.adj[k,i] != 0:
                outs.append(k)
        return outs
        #return np.ndarray.tolist(np.ndarray.astype(self.adj[:,i], int))
    
    # Reset weights
    def reset_weights(self):
        self.weights = np.copy(self.adj)
    
    # Initialize random edge weights, assuming edge weights are all 0 or 1
    def init_random(self):
        self.weights = np.random.rand(self.size, self.size)*self.adj
    
    # Normalize columns to satisfy Markov state transition rules
    def init_markov(self):
        self.init_random()
        for i in range(self.size):
            colsum = sum(self.weights[:,i])
            if colsum != 0:
                for j in range(self.size):
                    self.weights[j,i] /= colsum
    
    # Make the vector of indices be the input nodes
    def define_input(self, v):
        self.input_nodes = v
    
    # Make the vector of indices be the output nodes
    def define_output(self, v):
        self.output_nodes = v
    
    # Reset state
    def reset_state(self):
        self.states = [[0 for i in range(self.size)]]
        self.tempstates = [[0 for i in range(self.size)]]
    
    # Runs one time step of the network, with the given input values, and returns the current output values
    def step(self, input_vals):
        self.time += 1
        # tempstate = [self.states[-1][i] for i in range(self.size)]
        # Edit the inputs of the last state. This will make things consistent, so each step is a single state
        if self.mode == 'overwrite':
            for i in range(len(self.input_nodes)):
                self.states[-1][self.input_nodes[i]] = input_vals[i]
        if self.mode == 'add':
            for i in range(len(self.input_nodes)):
                self.states[-1][self.input_nodes[i]] += input_vals[i]
        # Weighted sum of inputs
        tempstate = np.dot(self.weights, self.states[-1])
        # Apply activation functions
        state = [tempstate[i] for i in range(self.size)]
        for i in range(self.size):
            #if self.act[i] == 'linear':
            #    do nothing
            if self.act[i] == 'relu':
                state[i] = state[i] if state[i] > 0 else 0
            if self.act[i] == 'sigmoid' or self.act[i] == 'logistic':
                state[i] = 1.0/(1.0+exp(-state[i]))
            if self.act[i] == 'tanh':
                state[i] = (exp(2*state[i]) - 1)/(exp(2*state[i]) + 1)
        output_vals = [state[self.output_nodes[i]] for i in range(len(self.output_nodes))]
        self.tempstates.append(tempstate)
        self.states.append(state)
        return output_vals
    
    # Runs several time steps of the network, with the given vector of input value vectors at each time step, 
    # and returns a vector of output values
    def steps(self, input_vals_v, n=0):
        output_vals_v = []
        empty_input = [0 for i in range(len(self.input_nodes))]
        for t in range(max(n, len(input_vals_v))):
            input_vals = empty_input
            if t < len(input_vals_v):
                input_vals = input_vals_v[t]
            output_vals_v.append(self.step(input_vals))
        return output_vals_v
    
    # Run n steps on a input only at initial step
    def stepn(self, input_vals_0, n=1):
        output_vals_v = []
        empty_input = [0 for i in range(len(self.input_nodes))]
        for t in range(n):
            input_vals = empty_input
            if t == 0:
                input_vals = input_vals_0
            output_vals_v.append(self.step(input_vals))
        return output_vals_v
    
    # Waits until the first nonzero output value and uses that as the real output
    def error_first(self, train_vals, label_vals, max_step=100):
        self.reset_state()
        output_vals = [0 for i in range(len(self.output_nodes))]
        empty_input = [0 for i in range(len(self.input_nodes))]
        t = 0
        while t < max_step and np.count_nonzero(output_vals) == 0:
            input_vals = empty_input
            if t == 0:
                input_vals = train_vals
            output_vals = self.step(input_vals)
            t += 1
        err_v = np.subtract(output_vals, label_vals)
        return err_v
    
    # Waits a fixed delay
    def error_delay(self, train_vals, label_vals, delay, reset=True, max_step=100):
        if reset:
            self.reset_state()
        output_vals = [0 for i in range(len(self.output_nodes))]
        empty_input = [0 for i in range(len(self.input_nodes))]
        t = 0
        while t < delay:
            input_vals = empty_input
            if t == 0:
                input_vals = train_vals
            output_vals = self.step(input_vals)
            t += 1
        err_v = np.subtract(output_vals, label_vals)
        return err_v
    
    # Mean square error of training batch. train_x and train_y are batches of training data
    # Data type options: 1_to_1...
    # If delay = 0, first nonzero output will be used, otherwise there will be a fixed delay before the output is sampled
    def mse_batch(self, train_x, train_y, data_type='1_to_1', delay=0, reset=True, max_step=100):
        mse = 0
        for k in range(len(train_x)):
            if data_type == '1_to_1':
                if delay == 0:
                    mse += sum(self.error_first(train_x[k], train_y[k], max_step)**2)
                else:
                    mse += sum(self.error_delay(train_x[k], train_y[k], delay, reset, max_step)**2)
        mse /= 2*len(train_x) # Factor of 1/2 for the coefficient of 2 on the derivative to cancel, as usual...
        return mse
    
    def backprop(self, train_vals, label_vals, delay):
        dw = np.zeros((self.size, self.size))
        # E = 1/2 (observed - expected)**2
        err_v = self.error_delay(train_vals, label_vals, delay)
        nodes_i = [self.output_nodes[i] for i in range(len(self.output_nodes))]
        nodes_d = [err_v[i] for i in range(len(self.output_nodes))]
        newnodes_i = []
        newnodes_d = []
        # dE/dw = (observed - expected) * d(observed)/dw
        for k in range(1, len(self.states)): # Or max step size, maybe? 
            for i in range(len(nodes_i)):
                # node value = activation( weighted sum of input values )
                # d node value / dw = activation'( weighted sum of input values ) * d (weighted sum of input values) / dw
                node_val = self.states[len(self.states)-k][nodes_i[i]]
                node_sum = self.tempstates[len(self.states)-k][nodes_i[i]]
                act_deriv = 1
                #if self.act[nodes_i[i]] == 'linear':
                #    do nothing
                if self.act[nodes_i[i]] == 'relu' and node_sum < 0:
                    act_deriv = 0
                if self.act[nodes_i[i]] == 'sigmoid' or self.act[nodes_i[i]] == 'logistic':
                    act_deriv = node_val * (1 - node_val)
                if self.act[nodes_i[i]] == 'tanh':
                    act_deriv = 4.0/(exp(node_sum)+exp(-node_sum))**2
                in_nodes_i = self.inputs(nodes_i[i])
                for j in range(len(in_nodes_i)):
                    # Self-weights are covered just fine when nodes_i[i] = in_nodes_i[j]
                    in_node_val = self.states[len(self.states)-(k+1)][in_nodes_i[j]]
                    # d (weighted sum of input values) / dw = input value [...for w in these weights]
                    dweight = nodes_d[i] * act_deriv * in_node_val
                    # Add this to the total derivative with respect to this weight
                    dw[nodes_i[i], in_nodes_i[j]] += dweight
                    print(in_node_val)
                    print(dw)
                    # nodes_d stores the buildup of chained derivatives
                    weight = self.weights[nodes_i[i], in_nodes_i[j]]
                    # d (weighted sum of input values) / dw = ... * weight * d node value / dw
                    if in_nodes_i[j] in newnodes_i:
                        newnodes_d[in_nodes_i[j]] += nodes_d[i] * act_deriv * weight
                    else:
                        newnodes_i.append(in_nodes_i[j])
                        newnodes_d.append( nodes_d[i] * act_deriv * weight )
            nodes_i = newnodes_i
            nodes_d = newnodes_d
            newnodes_i = []
            newnodes_d = []
        return dw

In [340]:
test = GraphNet(4)
test.connect(0,1)
test.connect(1,2)
test.connect(2,1)
test.connect(2,3)
test.define_input([0])
test.define_output([3])

In [341]:
test.adj
test.init_markov()
test.weights

array([[0.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.41474401, 0.        ],
       [0.        , 1.        , 0.        , 0.        ],
       [0.        , 0.        , 0.58525599, 0.        ]])

In [168]:
test.stepn([1], 10)

[[0.0],
 [0.0],
 [0.44203492419103935],
 [0.0],
 [0.2466400499864614],
 [0.0],
 [0.13761653418822178],
 [0.0],
 [0.07678521993089758],
 [0.0]]

In [172]:
test.reset_state()
test.steps([[1],[0.5],[0.25],[0.125]], 10)

[[0.0],
 [0.0],
 [0.44203492419103935],
 [0.22101746209551967],
 [0.35714878103422126],
 [0.17857439051711063],
 [0.19927654668483713],
 [0.09963827334241857],
 [0.11118935347795303],
 [0.055594676738976515]]

In [287]:
train_x = [[1]]
train_y = [[1]]
test.mse_batch(train_x, train_y, delay=3)

0.3423384182578477

In [345]:
test.backprop([1], [1], 5)

0.41474400636523623
[[ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.         -0.31407261  0.        ]]
0.41474400636523623
[[ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.         -0.18381288  0.          0.        ]
 [ 0.          0.         -0.31407261  0.        ]]
0
[[ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.         -0.18381288  0.          0.        ]
 [ 0.          0.         -0.31407261  0.        ]]
1.0
[[ 0.          0.          0.          0.        ]
 [ 0.          0.         -0.44319598  0.        ]
 [ 0.         -0.18381288  0.          0.        ]
 [ 0.          0.         -0.31407261  0.        ]]
1.0
[[ 0.          0.          0.          0.        ]
 [ 0.          0.         -0.44319598  0.        ]
 [ 0.         -0.36762575  0

array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [-0.18381288,  0.        , -0.44319598,  0.        ],
       [ 0.        , -0.36762575,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.31407261,  0.        ]])

In [346]:
test.error_delay([1], [1], 5)
test.states

[[1, 0, 0, 0],
 [0, 1.0, 0.0, 0.0],
 [0, 0.0, 1.0, 0.0],
 [0, 0.41474400636523623, 0.0, 0.5852559936347638],
 [0, 0.0, 0.41474400636523623, 0.0],
 [0.0, 0.1720125908158871, 0.0, 0.24273141554934913]]